# Identificação de toxicidade

In [ ]:
"""
Com base nos datasets de classificação textual e modelos de linguagem para a língua portuguesa 
listados na plataforma Huggingface, implemente um classificador textual para uma determinada tarefa.
Para tal, se baseie no tutorial linkado abaixo,

Atenção, sua implementação DEVE ser baseada na utilização das bibliotecas transformers e datasets, 
podendo utilizar como infraestrutura básica as bibliotecas Pytorch, Tensorflow ou keras.
"""

Esse notebook implementa um modelo de classificação de texto em português brasileiro para detecção de discurso de ódio. O modelo utilizado é o [xlm-roberta-base](https://huggingface.co/papluca/xlm-roberta-base-language-detection), que é um modelo pré-treinado para classificação de texto mulilingual. O modelo foi treinado com o dataset em português brasileiro [told-br](https://huggingface.co/datasets/JAugusto97/told-br), contendo tweets rotulados como tóxicos ou não tóxicos.

In [ ]:
!pip install -q datasets transformers[sentencepiece] langid watermark

In [89]:
import time
from pathlib import Path

import torch
from datasets import load_dataset
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline,
    Trainer,
    TrainingArguments
)

In [90]:
# Setando o caminho para o modelo
model_dir = Path('./model_checkpoints')

In [ ]:
# Carregando o dataset e definindo os datasets de treino, validação e teste
dataset = load_dataset("JAugusto97/told-br")

ds_train = dataset['train']
ds_valid = dataset['validation']
ds_test = dataset['test']

In [93]:
# Carregando o modelo e o tokenizador
model_ckpt = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize_text(sequence):
    """Tokenize input sequence."""
    return tokenizer(sequence["text"], truncation=True, max_length=128)

# Tokenizando os datasets de treino, validação e teste
tok_train = ds_train.map(tokenize_text, batched=True)
tok_valid = ds_valid.map(tokenize_text, batched=True)
tok_test = ds_test.map(tokenize_text, batched=True)

In [96]:
label2id = {0: 0, 1: 1}
id2label = {0: 0, 1: 1}

def encode_labels(example):
    """Map string labels to integers."""
    example["label"] = label2id[example["label"]]
    return example

tok_train = tok_train.map(encode_labels, batched=False)
tok_valid = tok_valid.map(encode_labels, batched=False)
tok_test = tok_test.map(encode_labels, batched=False)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Instanciando o modelo
model = AutoModelForSequenceClassification.from_pretrained(
  model_ckpt, num_labels=len([0, 1]), id2label=id2label, label2id=label2id
)

In [102]:
def compute_metrics(pred):
    """Custom metric to be used during training."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)  # Accuracy
    f1 = f1_score(labels, preds, average="weighted")  # F1-score
    return {
        "accuracy": acc,
        "f1": f1
        }

In [ ]:
epochs = 2
lr = 2e-5
train_bs = 64
eval_bs = train_bs * 2
logging_steps = len(tok_train) // train_bs
output_dir = model_dir / "xlm-roberta-base-finetuned-told-br"

training_args = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=epochs,
  learning_rate=lr,
  per_device_train_batch_size=train_bs,
  per_device_eval_batch_size=eval_bs,
  evaluation_strategy="epoch",
  logging_steps=logging_steps,
  fp16=True,
)

In [104]:
trainer = Trainer(
  model,
  training_args,
  compute_metrics=compute_metrics,
  train_dataset=tok_train,
  eval_dataset=tok_valid,
  data_collator=data_collator,
  tokenizer=tokenizer,
)

In [ ]:
# Treinando o modelo
trainer.train()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Salvando o modelo no Hugging Face Hub
trainer.push_to_hub()

In [109]:
ds_test = ds_test.to_pandas()
ds_test.head(3)

,text,label
0,@user nossa mano te odeio na moral kkkkkkkkkkk...,1
1,@user @user a edição ficou muito boa! kkkkkkkk...,0
2,"@user largada vá, visto que a vaca anda à solta 😂",0


In [ ]:
device = 0 if torch.cuda.is_available() else -1

# Carregando do huggingface o modelo treinado
model_ckpt = "joaollm/xlm-roberta-base-finetuned-told-br"
pipe = pipeline("text-classification", model=model_ckpt, device=device)

In [112]:
start_time = time.perf_counter()
model_preds = [s['label'] for s in pipe(ds_test.text.values.tolist(), truncation=True, max_length=128)]
print(f"{time.perf_counter() - start_time:.2f} seconds")

22.58 seconds


In [113]:
print(classification_report(ds_test.label.values.tolist(), model_preds, digits=3))

              precision    recall  f1-score   support

           0      0.823     0.703     0.758      1128
           1      0.705     0.825     0.761       972

    accuracy                          0.760      2100
   macro avg      0.764     0.764     0.760      2100
weighted avg      0.769     0.760     0.759      2100

